# Generating counterfactuals for multi-class classification and regression models
This notebook will demonstrate how the DiCE library can be used for multiclass classification and regression for scikit-learn models.
You can use any method ("random", "kdtree", "genetic"), just specific it in the method argument in the initialization step. The rest of the code is completely identical.
For demonstration, we will be using the genetic algorithm for CFs.

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import dice_ml
from dice_ml import Dice

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

We will use sklearn's internal datasets to demonstrate DiCE's features in this notebook

In [5]:
outcome_name = 'target'
# Function to process sklearn's internal datasets
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df[outcome_name] = pd.Series(sklearn_dataset.target)
    return df

## Multiclass Classification

For multiclass classification, we will use sklearn's Iris dataset. This data set consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset

In [6]:
from sklearn.datasets import load_iris
df_iris = sklearn_to_df(load_iris())
df_iris.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [7]:
df_iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
target               150 non-null int64
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [8]:
continuous_features_iris = df_iris.drop(outcome_name, axis=1).columns.tolist()
target = df_iris[outcome_name]

In [9]:
# Split data into train and test
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

datasetX = df_iris.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

categorical_features = x_train.columns.difference(continuous_features_iris) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_iris),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf_iris = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model_iris = clf_iris.fit(x_train, y_train)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [10]:
d_iris = dice_ml.Data(dataframe=df_iris, 
                      continuous_features=continuous_features_iris, 
                      outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
m_iris = dice_ml.Model(model=model_iris, backend="sklearn", model_type='classifier')

In [11]:
exp_genetic_iris = Dice(d_iris, m_iris, method="genetic")

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/preprocessing/_function_transformer.py:97: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)


ValueError: Specifying the columns using strings is only supported for pandas DataFrames

As we can see below, all the target values will lie in the desired class

In [1]:
# Single input 
query_instances_iris = x_train[2:3]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=4, desired_class = 2)
genetic_iris.visualize_as_dataframe()

NameError: name 'x_train' is not defined

In [11]:
# Multiple queries can be given as input at once
query_instances_iris = x_train[17:19]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=7, desired_class = 2)
genetic_iris.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.7,2.9,4.2,1.3,1



Diverse Counterfactual set (new outcome: 2)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,7.8,3.1,4.1,1.8,2.0
1,7.3,3.2,6.5,1.7,2.0
2,7.0,-,5.2,2.0,2.0
3,5.4,3.3,6.0,1.8,2.0
4,6.5,2.5,3.7,1.9,2.0
5,4.6,3.1,6.1,2.0,2.0
6,-,3.0,2.9,2.0,2.0


Query instance (original outcome : 1)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,6.3,3.3,4.7,1.6,1



Diverse Counterfactual set (new outcome: 2)


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.8,3.6,5.0,2.3,2.0
1,5.0,4.1,5.7,2.1,2.0
2,6.4,4.0,4.2,1.9,2.0
3,6.2,3.6,5.7,-,2.0
4,5.7,3.2,6.8,1.7,2.0
5,6.2,3.0,2.1,2.0,2.0
6,7.6,3.5,1.3,2.4,2.0


# Regression

For regression, we will use sklearn's boston dataset. This dataset contains boston house-prices. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset

In [12]:
from sklearn.datasets import load_boston
df_boston = sklearn_to_df(load_boston())
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [13]:
df_boston.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     506 non-null    float64
 1   ZN       506 non-null    float64
 2   INDUS    506 non-null    float64
 3   CHAS     506 non-null    float64
 4   NOX      506 non-null    float64
 5   RM       506 non-null    float64
 6   AGE      506 non-null    float64
 7   DIS      506 non-null    float64
 8   RAD      506 non-null    float64
 9   TAX      506 non-null    float64
 10  PTRATIO  506 non-null    float64
 11  B        506 non-null    float64
 12  LSTAT    506 non-null    float64
 13  target   506 non-null    float64
dtypes: float64(14)
memory usage: 55.5 KB


In [14]:
continuous_features_boston = df_boston.drop(outcome_name, axis=1).columns.tolist()
target = df_boston[outcome_name]

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
# Split data into train and test
datasetX = df_boston.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0)

categorical_features = x_train.columns.difference(continuous_features_boston) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_boston),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
regr_boston = Pipeline(steps=[('preprocessor', transformations),
                      ('regressor', RandomForestRegressor())])
model_boston = regr_boston.fit(x_train, y_train)

In [16]:
d_boston = dice_ml.Data(dataframe=df_boston, continuous_features=continuous_features_boston, outcome_name=outcome_name)
# We provide the type of model as a parameter (model_type)
m_boston = dice_ml.Model(model=model_boston, backend="sklearn", model_type='regressor')

In [17]:
exp_genetic_boston = Dice(d_boston, m_boston, method="genetic")

As we can see below, all the target values will lie in the desired range

In [18]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[2:3]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston, 
                                                             total_CFs=2, 
                                                             desired_range=[30, 45])
genetic_boston.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 24)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.11329,30.0,4.93,0.0,0.428,6.897,54.3,6.3361,6.0,300.0,16.6,391.25,11.38,23.957



Diverse Counterfactual set (new outcome: [30, 45])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,21.58873,12.2,9.4,0.6,0.545,8.733,9.9,7.9497,10.1,400.8,14.3,172.8,9.76,38.17599868774414
1,5.78773,41.1,18.1,0.5,0.518,8.122,30.7,8.8876,8.2,488.1,21.5,296.4,5.31,36.941001892089844


In [19]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[17:19]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston, total_CFs=4, desired_range=[40, 50])
genetic_boston.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 49)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.01501,90.0,1.21,1.0,0.401,7.923,24.8,5.885,1.0,198.0,13.6,395.52,3.16,49.165



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,6.08761,89.0,14.9,0.7,0.662,7.776,81.4,9.3139,19.0,372.2,12.8,164.2,6.19,42.11199951171875
1,2.42374,66.7,14.0,0.6,0.44,8.643,12.0,2.9876,3.2,222.0,19.3,35.9,2.71,40.66299819946289
2,38.18897,12.8,23.3,0.3,0.704,8.239,32.6,12.0539,17.6,302.2,15.9,392.8,4.82,41.849998474121094
3,38.78235,75.1,19.9,-,0.658,8.658,3.8,4.8621,23.5,622.5,18.2,307.3,3.51,40.07500076293945


Query instance (original outcome : 31)


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.06911,45.0,3.44,0.0,0.437,6.739,30.8,6.4798,5.0,398.0,15.2,389.71,4.69,30.673



Diverse Counterfactual set (new outcome: [40, 50])


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,44.40979,22.9,0.5,0.6,0.444,7.863,11.5,10.386,22.2,442.5,13.7,371.3,3.54,44.891998291015625
1,6.65754,8.0,27.0,0.2,0.617,8.44,37.1,6.5442,19.9,680.3,-,284.5,3.16,43.34600067138672
2,55.29387,31.0,6.6,0.3,0.581,8.38,65.1,10.9898,2.6,593.7,12.9,346.4,4.53,45.277000427246094
3,1.67778,96.0,18.9,0.5,0.668,8.164,76.6,3.7475,16.0,461.1,14.8,92.5,2.92,45.29899978637695
